# Chapter 3 클래스와 상속

## Better way 22 딕셔너리와 튜플보다는 헬퍼 클래스로 관리하자

In [8]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, score):
        self._grades[name].append(score)
        
    def avereage_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

In [9]:
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)

print(book.avereage_grade('Isaac Newton'))

90.0


사전은 과도하게 쓰면 코드를 취약하게 작성할 위험이 있다.

In [10]:
class BySubjectGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append(grade)
    
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

In [11]:
book = BySubjectGradebook()
book.add_student('thkwon')
book.report_grade('thkwon', 'math', 100)
book.report_grade('thkwon', 'math', 90)
book.report_grade('thkwon', 'gym', 90)
book.report_grade('thkwon', 'gym', 80)
print(book.average_grade('thkwon'))

90.0


In [15]:
grades = []
grades.append((95, 0.45))
# ...

total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight

In [16]:
average_grade

95.0

In [17]:
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))

In [25]:
class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [30]:
class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [31]:
class Gradebook(object):
    def __init__(self):
        self._students = {}
        
    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

In [35]:
book = Gradebook()
albert = book.student('Albert Einstein')
math = albert.subject('math')
math.report_grade(80, 0.10)
math.report_grade(90, 0.10)
math.report_grade(74, 0.2)

print(albert.average_grade())

79.5


In [36]:
from collections import namedtuple

pin_info = namedtuple('card', 'name age phone_num')
thkwon = pin_info('thkwon', 30, '010-7331-4120')

In [39]:
thkwon.name

'thkwon'

In [40]:
thkwon.age

30

In [41]:
thkwon.phone_num

'010-7331-4120'

In [44]:
thkwon[0] + " " + str(thkwon[1]) + " " + thkwon[2]

'thkwon 30 010-7331-4120'

## Better way 15 클로저가 변수 스코프와 상호 작용하는 방법을 알자

In [11]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (1, x)
        return (2,x)
    values.sort(key=helper)
    
numbers = [8,3,1,2,5,4,7,6]
group = {2,3,5,7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


In [16]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            print(found)
            return (0, x)
        return (1, x)
    print("--out1--")
    print(found)
    print("--out2--")
    numbers.sort(key=helper)
    return found

In [17]:
found = sort_priority2(numbers, group)
print('Found:', found)
print(numbers)

--out1--
False
--out2--
True
True
True
True
Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


In [22]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [24]:
found = sort_priority3(numbers, group)
print('Found:', found)
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


In [29]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
       	return (1, x)
    
    
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True

## Better way 23 인터페이스가 간단하면 클래스 대신 함수를 받자

In [52]:
names = ['Kwon','Tae','Hyoung','hohoho']
names.sort(key=lambda x: len(x))
print(names)

['Tae', 'Kwon', 'Hyoung', 'hohoho']


In [56]:
from collections import defaultdict

def log_missing():
    print('Key added')
    return 0

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]
# log_missing은 defaultdict의 초기값을 지정하는 인자   
result = defaultdict(log_missing, current)
print('Befor: ', dict(result))

for key, amount in increments:
    result[key] += amount
    
print('After: ', dict(result))


Befor:  {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


In [86]:
current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 15),
    ('orange', 9),
    ('black', 7),
]

def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)

    for key, amount in increments:
        result[key] += amount
        
    return result, added_count

In [88]:
result,count = increment_with_report(current, increments)
assert count == 3

In [93]:
class CountMissing(object):
    def __init__(self):
        self.added = 0
        
    def missing(self):
        self.added += 1
        return 0

counter = CountMissing()
result = defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount
assert counter.added == 3

In [100]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0

counter = BetterCountMissing()
result = defaultdict(counter, current)
for key, amount in increments:
    result[key] += amount
print(counter.added)
assert counter.added == 3

3
